In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from imblearn.over_sampling import SMOTE
%matplotlib inline

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn import metrics 
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import ndcg_score
from sklearn.metrics import average_precision_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import recall_score, precision_score

from sklearn.tree import DecisionTreeClassifier

import yaml
from recommenders.datasets.python_splitters import python_random_split

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from math import sqrt

import unittest
from unittest import mock

import pandas as pd
from recmetrics import metrics
from typing import List
import numpy as np

import time
from sklearn.metrics import make_scorer, mean_squared_error
import lightgbm as lgb
import scipy.sparse as sp
from sklearn.metrics.pairwise import cosine_similarity

import math

from tqdm import tqdm
from surprise.model_selection import KFold
from sklearn.model_selection import StratifiedKFold


from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

from typing import List
from math import sqrt
import itertools

from sklearn.metrics import precision_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold


In [2]:
#POTREBNO IZMJENIT PUTANJU!!!!

#učitaj generirane podatke iz dataseta
playlist_tracks_df_1 = pd.read_pickle(r"C:\Users\Ivan\PycharmProjects\sars4\sars4\spotify\playlist_tracks.pkl")
playlist_tracks_df_2 = pd.read_pickle(r"C:\Users\Ivan\PycharmProjects\sars8\spotify\playlist_tracks_2.pkl")
playlist_tracks_df_3 = pd.read_pickle(r"C:\Users\Ivan\PycharmProjects\sars10\spotify\playlist_tracks_3.pkl")
playlist_tracks_df = pd.concat([playlist_tracks_df_1,playlist_tracks_df_2, playlist_tracks_df_3], ignore_index=True)

In [3]:
set_playlist_unique = set(playlist_tracks_df['playlist_id'].unique()) #SKUP JEDINSTVENIH PLAYLISTA
playlist_tracks_df['isin']=0                                          

In [4]:
#NAPRAVI DICTIONARY SVIH PLAYLIST NA KOJEM JE SE S 1 OZNAČAVA SAMO PLAYLISTA KOJA JE KEY U DICTIONARYJU

i=1
playlist_tracks_df_dict={}
for playlist in set_playlist_unique:                                                      
    copy_playlist_tracks_df=playlist_tracks_df.copy()
    variable_name = f"playlist_tracks_df{i}"                                           
    copy_playlist_tracks_df.loc[copy_playlist_tracks_df['playlist_id']==playlist, 'isin'] = 1      #stavi 1 gdje se nalazi u toj playlist
    playlist_tracks_df_dict[variable_name] = copy_playlist_tracks_df
    i+=1

In [5]:
#NAPRAVI PREDVIĐANJA 

X_train_dict={}
y_train_dict={}
X_test_dict={}
y_test_dict={}


for playlist in playlist_tracks_df_dict:
    print(playlist)
    X_train_name = playlist
    y_train_name = playlist
    X_test_name = playlist
    y_test_name = playlist

    train_set, test_set = python_random_split(playlist_tracks_df_dict[playlist], 0.8)

    X = train_set.drop(columns=['genres','album_genres','added_by','album_release_date','added_at','playlist_id','playlist_tracks','playlist_name','album_type','album_tracks','artist_name','album_artist_id','album_artist_name','album_id','album_name','id', 'name','type','is_local','explicit','disc_number','track_number','artist_id','uri','track_href','analysis_url','isin'])
    y = train_set['isin']

    X_train=X.copy()
    y_train=y.copy()
    X_test = test_set.drop(columns=['genres','album_genres','added_by','album_release_date','added_at','playlist_id','playlist_tracks','playlist_name','album_type','album_tracks','artist_name','album_artist_id','album_artist_name','album_id','album_name','id', 'name','type','is_local','explicit','disc_number','track_number','artist_id','uri','track_href','analysis_url','isin'])
    y_test = test_set['isin']
        
    X_train_dict[X_train_name]=X_train.copy()
    y_train_dict[y_train_name]=y_train.copy()
    X_test_dict[X_test_name]=X_test.copy()
    y_test_dict[y_test_name]=y_test.copy()

    pipex = make_pipeline(StandardScaler(), DecisionTreeClassifier(max_depth=30))                         #NAPRAVI PIPELINE
    pipex.fit(X_train, y_train)                                                                           #UCI MODEL
    Pipeline(steps=[('standardscaler', StandardScaler()),('dt', DecisionTreeClassifier(max_depth=30))])   #DEFINIRAJ METODU I SKALIRANJE ZA PIPELINE
    pipex.score(X_test, y_test)

    #NAPRAVI PREDVIĐANJA
    prob_preds = pipex.predict_proba(playlist_tracks_df_dict[playlist].drop(['genres','album_genres','added_by','album_release_date','added_at','playlist_id','playlist_tracks','playlist_name','album_type','album_tracks','artist_name','album_artist_id','album_artist_name','album_id','album_name','id', 'name','type','is_local','explicit','disc_number','track_number','artist_id','uri','track_href','analysis_url','isin'], axis=1))

    #SMJESTI DOBIVENA PREDVIĐANJA U DATAFRAME
    for item in range(len(prob_preds)):
        playlist_tracks_df_dict[playlist].at[item, 'prediction']=prob_preds[item][1]
    
    print("------------------------------------------------------")

playlist_tracks_df1
------------------------------------------------------
playlist_tracks_df2
------------------------------------------------------
playlist_tracks_df3
------------------------------------------------------
playlist_tracks_df4
------------------------------------------------------
playlist_tracks_df5
------------------------------------------------------
playlist_tracks_df6
------------------------------------------------------
playlist_tracks_df7
------------------------------------------------------
playlist_tracks_df8
------------------------------------------------------
playlist_tracks_df9
------------------------------------------------------
playlist_tracks_df10
------------------------------------------------------
playlist_tracks_df11
------------------------------------------------------
playlist_tracks_df12
------------------------------------------------------
playlist_tracks_df13
------------------------------------------------------
playlist_tracks_df14


------------------------------------------------------
playlist_tracks_df110
------------------------------------------------------
playlist_tracks_df111
------------------------------------------------------
playlist_tracks_df112
------------------------------------------------------
playlist_tracks_df113
------------------------------------------------------
playlist_tracks_df114
------------------------------------------------------
playlist_tracks_df115
------------------------------------------------------
playlist_tracks_df116
------------------------------------------------------
playlist_tracks_df117
------------------------------------------------------
playlist_tracks_df118
------------------------------------------------------
playlist_tracks_df119
------------------------------------------------------
playlist_tracks_df120
------------------------------------------------------
playlist_tracks_df121
------------------------------------------------------
playlist_tracks_df122

In [6]:
#IZRACUNAJ PROSJECNI STOTI ELEMENT

average_100th_list=[]
for user in tqdm(playlist_tracks_df_dict):
    average_100th_list.append(playlist_tracks_df_dict[user]['prediction'].sort_values(ascending=False).iloc[99])
average_100th=sum(average_100th_list)/len(average_100th_list)
print("AVERAGE 100TH:  ",average_100th)

100%|███████████████████████████████████████████████████████████████████████████████| 149/149 [00:00<00:00, 614.60it/s]

AVERAGE 100TH:   0.2291240196713466


In [7]:
#Izračunaj koliko se koja pjesma ponavlja u preporukama za sve korisnike

preds_dict_rec_100={}
items_count_dict={}
for elem in playlist_tracks_df.index:
    items_count_dict[elem]=0
p=0
for playlist in playlist_tracks_df_dict:
    preds_dict_rec_100[playlist]=playlist_tracks_df_dict[playlist]['prediction'].sort_values(ascending=False).head(99)
    for item in preds_dict_rec_100[playlist].index:
        items_count_dict[item]+=1
    p+=1

In [8]:
print("NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION-NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION")

NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION-NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION


In [9]:
#EVALUACIJA NOVELTY
def novelty(predicted: dict, pop: dict, u: int, n: int) -> (float, list):
    
    mean_self_information = []
    k = 0
    for pred25 in predicted: 
        self_information = 0
        k += 1
        for track_index in predicted[pred25].index:
            self_information += np.sum(-np.log2(pop[track_index]/u))
        mean_self_information.append(self_information/n)
    novelty = sum(mean_self_information)/k
    return novelty, mean_self_information
    
test_users = len(playlist_tracks_df_dict) # Total unique users from notebook demo
test_recs_per_user = 100
# WHEN metrics.novelty is run
novelty_score, _ = novelty(
            predicted = preds_dict_rec_100,#predicted data
            pop = items_count_dict, #test data
            u = test_users, #broj korisnika
            n = test_recs_per_user #broj testnih recorda
)

print("NOVELTY SCORE:   ",novelty_score)

NOVELTY SCORE:    5.96590159175412


In [10]:
#EVALUACIJA CATALOG COVERAGE
def catalog_coverage(catalog: dict) -> float:
    sum_non_zero=0
    for elem in catalog:
        if(catalog[elem]!=0):
            sum_non_zero+=1
    catalog_coverage = round(sum_non_zero/(len(catalog)*1.0),2)
    return catalog_coverage
catalog_coverage = catalog_coverage(
    catalog = items_count_dict, #test data
)

print("CATALOG COVERAGE SCORE:   ",catalog_coverage)


CATALOG COVERAGE SCORE:    0.85


In [11]:
#EVALUACIJA PERSONALIZATION

def personalization(predicted: List[list]) -> float:

    def make_rec_matrix(predicted: List[list]) -> sp.csr_matrix:
        df = pd.DataFrame(data=predicted).reset_index().melt(
            id_vars='index', value_name='item',
        )
        df = df[['index', 'item']].pivot(index='index', columns='item', values='item')
        df = pd.notna(df)*1
        rec_matrix = sp.csr_matrix(df.values)
        return rec_matrix

    #create matrix for recommendations
    predicted = np.array(predicted)
    rec_matrix_sparse = make_rec_matrix(predicted)

    #calculate similarity for every user's recommendation list
    similarity = cosine_similarity(X=rec_matrix_sparse, dense_output=False)

    #calculate average similarity
    dim = similarity.shape[0]
    personalization = (similarity.sum() - dim) / (dim * (dim - 1))
    return 1-personalization

pred100_list = [preds_dict_rec_100[playlist].index.tolist() for playlist in preds_dict_rec_100]

# WHEN metrics.personalization is run
personalization_score =personalization(
    predicted=pred100_list)

# THEN the personalization score should be within the expected value
print("PERSONALIZATION SCORE:    ",personalization_score)

PERSONALIZATION SCORE:     0.9805959101261115


In [12]:
#RAZLICITOST

def jaccard_similarity(dframe, average_100th):
    index_rating = dframe[dframe['isin']==0].index
    index_pred = dframe[dframe['prediction']>average_100th].index
    intersection_size = len(index_rating[index_rating.isin(index_pred)])
    union_size = len(index_rating.union(index_pred))
    return intersection_size / union_size if union_size != 0 else 0.0

def inter_list_diversity(dframe, average_100th):
    length_frame = len(dframe)
    pairwise_similarities = []    
    similarity = jaccard_similarity(dframe, average_100th)    
    return similarity

diversity_score_list=[]
for user in tqdm(playlist_tracks_df_dict):
    diversity_score_list.append(inter_list_diversity(playlist_tracks_df_dict[user],average_100th))
print("Inter-List Diversity Score:", np.mean(diversity_score_list))

100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [00:06<00:00, 24.35it/s]

Inter-List Diversity Score: 0.0038817228145979063


In [17]:
print("CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-")

CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-


In [18]:
#UCI MODEL

j=1
dt_dict={}
for playlist in tqdm(playlist_tracks_df_dict): 
    dt_dict[playlist] = DecisionTreeClassifier(max_depth=30).fit(X_train_dict[playlist], y_train_dict[playlist])
    j+=1

100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [00:36<00:00,  4.05it/s]


In [20]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_list_cv=[]
for playlist in tqdm(playlist_tracks_df_dict): 
    rmse_scorer = make_scorer(rmse)
    rmse_results = cross_val_score(dt_dict[playlist], X_train_dict[playlist], y_train_dict[playlist],
                                   scoring=rmse_scorer, cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42))
    rmse_list_cv.append(np.mean(np.abs(rmse_results)))

cvscore_rmse=np.mean(rmse_list_cv)
print("CV cross_val_score RMSE:", cvscore_rmse)

  1%|█                                                                                 | 2/149 [00:02<03:08,  1.28s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  UserWarning,
 10%|████████▏                                                                        | 15/149 [00:15<02:13,  1.00it/s]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  UserWarning,
 41%|█████████████████████████████████▏                                               | 61/149 [01:11<01:44,  1.19s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits

CV cross_val_score RMSE: 0.09478307183116659


In [21]:
def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

mae_list_cv=[]
for playlist in tqdm(playlist_tracks_df_dict): 
    mae_scorer = make_scorer(mae)
    mae_results = cross_val_score(dt_dict[playlist], X_train_dict[playlist], y_train_dict[playlist],
                                  scoring=mae_scorer, cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42))
    mae_list_cv.append(np.mean(np.abs(mae_results)))

cvscore_mae=np.mean(mae_list_cv)
print("CV cross_val_score MAE:", cvscore_mae)

  1%|█                                                                                 | 2/149 [00:02<02:42,  1.10s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  UserWarning,
 10%|████████▏                                                                        | 15/149 [00:17<03:08,  1.41s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  UserWarning,
 41%|█████████████████████████████████▏                                               | 61/149 [01:13<01:38,  1.11s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits

CV cross_val_score MAE: 0.010726661471644817


In [22]:
def ndcg_scorer(y_true, y_score):
    ndcg=ndcg_score(y_true.values.reshape(1, -1), pd.Series(y_score).values.reshape(1, -1))
    return ndcg

ndcg_list_cv=[]
for playlist in tqdm(playlist_tracks_df_dict): 
    ndcg_scorer.__name__ = 'ndcg_scorer'
    ndcg_scores = cross_val_score(dt_dict[playlist], X_train_dict[playlist], y_train_dict[playlist],
                                  cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42),
                                  scoring=make_scorer(ndcg_scorer))
    ndcg_list_cv.append(np.mean(np.abs(ndcg_scores)))

print("CROSS_VAL_SCORE NDCG:", np.mean(ndcg_list_cv))

  1%|█                                                                                 | 2/149 [00:02<03:31,  1.44s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  UserWarning,
 10%|████████▏                                                                        | 15/149 [00:18<02:17,  1.03s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  UserWarning,
 41%|█████████████████████████████████▏                                               | 61/149 [01:15<01:45,  1.20s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits

CROSS_VAL_SCORE NDCG: 0.24227901361131557


In [23]:
def map_scorer(y_true, y_score):
    return average_precision_score(y_true.values, pd.Series(y_score).values)

map_cv_scores=[]
for playlist in tqdm(playlist_tracks_df_dict):
    if playlist!='playlist_tracks_df11111':
        df = playlist_tracks_df_dict[playlist].copy()
        map_scorer.__name__ = 'map_scorer'
        cv_map_scores = cross_val_score(dt_dict[playlist], X_train_dict[playlist], y_train_dict[playlist],
                                        scoring=make_scorer(map_scorer),
                                       cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42))
        for elem in cv_map_scores:
            if not math.isnan(elem):
                map_cv_scores.append(elem)

print("CROSS_VAL_SCORE Mean Average Precision:", np.mean(map_cv_scores))

  1%|█                                                                                 | 2/149 [00:05<06:08,  2.51s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  UserWarning,
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864

CROSS_VAL_SCORE Mean Average Precision: 0.024492898695624755


In [24]:
def precision_scorer(y_true, y_score):
    return (precision_score(y_true, pd.Series(y_score), average='macro'))

precision_list_cv=[]
for playlist in tqdm(playlist_tracks_df_dict):
    precision_scorer.__name__ = 'precision_scorer'

    precision_scores = cross_val_score(dt_dict[playlist], X_train_dict[playlist], y_train_dict[playlist], 
                        cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42), scoring=make_scorer(precision_scorer))
    precision_list_cv.append(np.mean((precision_scores)))

cvscore_precision=np.mean(precision_list_cv)
print("CV cross_val_score PRECISION:", cvscore_precision)

  0%|                                                                                          | 0/149 [00:00<?, ?it/s]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  1%|█                                                                                 | 2/149 [00:02<03:07,  1.28s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  UserWarning,
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted s

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 12%|█████████▊                                                                       | 18/149 [00:20<02:31,  1.16s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Us

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\i

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 32%|█████████████████████████▌                                                       | 47/149 [01:03<02:37,  1.55s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Us

 36%|█████████████████████████████▎                                                   | 54/149 [01:13<02:13,  1.40s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 37%|█████████████████████████████▉                                                   | 55/149 [01:14<02:02,  1.31s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification

...

 76%|████████████████████████████████████████████████████████████▋                   | 113/149 [02:26<00:45,  1.26s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Us

 84%|███████████████████████████████████████████████████████████████████             | 125/149 [02:42<00:29,  1.24s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Us

 89%|███████████████████████████████████████████████████████████████████████▍        | 133/149 [02:51<00:17,  1.11s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Us

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 99%|█████

CV cross_val_score PRECISION: 0.5291711906765965


In [25]:
def recall_scorer(y_true, y_score):
    return (recall_score(y_true, pd.Series(y_score), average='macro'))

recall_list_cv=[]
for playlist in tqdm(playlist_tracks_df_dict):
    recall_scores = cross_val_score(dt_dict[playlist], X_train_dict[playlist], y_train_dict[playlist], 
                        cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42), scoring=make_scorer(recall_scorer))
    recall_list_cv.append(np.mean((recall_scores)))
    
cvscore_recall=np.mean(recall_list_cv)
print("CV cross_val_score RECALL:", cvscore_recall)

  1%|█                                                                                 | 2/149 [00:02<02:49,  1.15s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  UserWarning,
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 10%|████████▏                                                                        | 15/149 [00:17<02:34,  1.15s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  UserWarning

CV cross_val_score RECALL: 0.5261920192453596


In [26]:
cvscore_f1=(2*cvscore_precision*cvscore_recall)/(cvscore_precision+cvscore_recall)
print("CV cross_val_score F1:", (cvscore_f1))

CV cross_val_score F1: 0.5276774000283381


In [27]:
print("CV cross_val_score RMSE:  ", str(cvscore_rmse))
print("CV cross_val_score MAE:  ", str(cvscore_mae))
print('CV cross_val_score PRECISION:  ' + str(cvscore_precision))
print('CV cross_val_score RECALL:  ' + str(cvscore_recall))
print('CV cross_val_score F1:  ' + str(cvscore_f1))
print("CV cross_val_score NDCG:", np.mean(ndcg_list_cv))
print("CV cross_val_score MAP:", np.mean(map_cv_scores))


CV cross_val_score RMSE:   0.09478307183116659
CV cross_val_score MAE:   0.010726661471644817
CV cross_val_score PRECISION:  0.5291711906765965
CV cross_val_score RECALL:  0.5261920192453596
CV cross_val_score F1:  0.5276774000283381
CV cross_val_score NDCG: 0.24227901361131557
CV cross_val_score MAP: 0.024492898695624755


In [18]:
print("STRATIFIED KFOLD-STRATIFIED KFOLD-STRATIFIED KFOLD-STRATIFIED KFOLD-STRATIFIED KFOLD-STRATIFIED KFOLD-STRATIFIED KFOLD")

STRATIFIED KFOLD-STRATIFIED KFOLD-STRATIFIED KFOLD-STRATIFIED KFOLD-STRATIFIED KFOLD-STRATIFIED KFOLD-STRATIFIED KFOLD


In [13]:
from sklearn.utils import shuffle

rmse_list_cv3=[]
mae_list_cv3=[]
map_list_cv3 = []
ndcg_list_cv3 = []

precision_list_cv3_micro = []
precision_list_cv3_macro = []
precision_list_cv3_weighted = []

recall_list_cv3_micro = []
recall_list_cv3_macro = []
recall_list_cv3_weighted = []

for playlist in tqdm(playlist_tracks_df_dict): 
    if (playlist!='playlist_tracks_df1111'):
        
        #STRATIFIEDKFOLD I PODJELA NA 10 DIJELOVA ZA EVALUACIJU
        
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        df = playlist_tracks_df_dict[playlist].copy()
        df = shuffle(df, random_state=42)  # Shuffle the data
        
        for fold, (train_idx, test_idx) in enumerate(skf.split(df, df['isin'])):        
            trainset = df.iloc[train_idx]
            testset = df.iloc[test_idx]

            #PONAVLJANJE GORE OBJASNJENOG PROCESA PREDVIDANJA
            
            X_tt = trainset.drop(columns=['genres','album_genres','added_by','album_release_date','added_at','playlist_id','playlist_tracks','playlist_name','album_type','album_tracks','artist_name','album_artist_id','album_artist_name','album_id','album_name','id', 'name','type','is_local','explicit','disc_number','track_number','artist_id','uri','track_href','analysis_url','isin'])
            y_tt = trainset['isin']

            X_train_tt=X_tt.copy()
            y_train_tt=y_tt.copy()

            X_test_tt = testset.drop(columns=['genres','album_genres','added_by','album_release_date','added_at','playlist_id','playlist_tracks','playlist_name','album_type','album_tracks','artist_name','album_artist_id','album_artist_name','album_id','album_name','id', 'name','type','is_local','explicit','disc_number','track_number','artist_id','uri','track_href','analysis_url','isin'])
            y_test_tt = testset['isin']

            clf_cv3 = DecisionTreeClassifier(max_depth=30)
            clf_cv3.fit(X_train_tt, y_train_tt)                                  #UCI MODEL
            y_probabilities = clf_cv3.predict_proba(X_test_tt)                   #NAPRAVI PREDVIDANJA

            preds_test_series = pd.DataFrame(columns=['true', 'prediction'])
            preds_eval_list=[]
            for i in range(len(y_probabilities)):
                preds_eval_list.append(y_probabilities[i][1])                    
            preds_test_series['true'] = y_test_tt
            preds_test_series['prediction'] = pd.Series(preds_eval_list).values

            #EVALUACIJA
            
            rmse_list_cv3.append(sqrt(mean_squared_error(preds_test_series['prediction'], preds_test_series['true'])))            
            mae_list_cv3.append((mean_absolute_error(preds_test_series['prediction'], preds_test_series['true'])))
            
            average_map = average_precision_score(preds_test_series['true'], preds_test_series['prediction'])
            if not math.isnan(average_map):
                map_list_cv3.append(average_map)

            average_ndcg=ndcg_score(preds_test_series['true'].values.reshape(1, -1), 
                                    preds_test_series['prediction'].values.reshape(1, -1))
            ndcg_list_cv3.append(average_ndcg)

            test_df=preds_test_series.copy()
            test_df.loc[test_df['prediction']>=average_100th, 'prediction']=1
            test_df.loc[test_df['prediction']<average_100th, 'prediction']=0
            
            precisions_micro=(precision_score(test_df['true'], test_df['prediction'], average='micro'))
            precision_list_cv3_micro.append(precisions_micro)
                
            precisions_macro=(precision_score(test_df['true'], test_df['prediction'], average='macro'))
            precision_list_cv3_macro.append(precisions_macro)
                
            precisions_weighted=(precision_score(test_df['true'], test_df['prediction'], average='weighted'))
            precision_list_cv3_weighted.append(precisions_weighted)
                
            recalls_micro=recall_score(test_df['true'], test_df['prediction'], average='micro')
            recall_list_cv3_micro.append(recalls_micro)
                
            recalls_macro=recall_score(test_df['true'], test_df['prediction'], average='macro')
            recall_list_cv3_macro.append(recalls_macro)
                
            recalls_weighted=recall_score(test_df['true'], test_df['prediction'], average='weighted')
            recall_list_cv3_weighted.append(recalls_weighted)

            
mean_rmse_score_cv3 = np.mean(rmse_list_cv3)
mean_mae_score_cv3 = np.mean(mae_list_cv3)
mean_map_score_cv3 = np.mean(map_list_cv3)
mean_ndcg_score_cv3 = np.mean(ndcg_list_cv3)

mean_precision_score_cv3_micro = np.mean(precision_list_cv3_micro)
mean_precision_score_cv3_macro = np.mean(precision_list_cv3_macro)
mean_precision_score_cv3_weighted = np.mean(precision_list_cv3_weighted)

mean_recall_score_cv3_micro = np.mean(recall_list_cv3_micro)
mean_recall_score_cv3_macro = np.mean(recall_list_cv3_macro)
mean_recall_score_cv3_weighted = np.mean(recall_list_cv3_weighted)

f1_score_cv3_micro=(2*mean_precision_score_cv3_micro*mean_recall_score_cv3_micro)/(mean_precision_score_cv3_micro+mean_recall_score_cv3_micro)
f1_score_cv3_macro=(2*mean_precision_score_cv3_macro*mean_recall_score_cv3_macro)/(mean_precision_score_cv3_macro+mean_recall_score_cv3_macro)
f1_score_cv3_weighted=(2*mean_precision_score_cv3_weighted*mean_recall_score_cv3_weighted)/(mean_precision_score_cv3_weighted+mean_recall_score_cv3_weighted)

print("CV SKLEARN.METRICS RMSE:    ", mean_rmse_score_cv3)
print("CV SKLEARN.METRICS MAE :    ", mean_mae_score_cv3)
print("CV SKLEARN.METRICS MAP:    ", mean_map_score_cv3)
print("CV SKLEARN.METRICS NDCG:    ", mean_ndcg_score_cv3)
print("CV SKLEARN.METRICS PRECISION:    ", mean_precision_score_cv3_micro, "|", mean_precision_score_cv3_macro, "|",
                                             mean_precision_score_cv3_weighted)
print("CV SKLEARN.METRICS RECALL:    ", mean_recall_score_cv3_micro, "|", mean_recall_score_cv3_macro, "|",
                                             mean_recall_score_cv3_weighted)

print("CV SKLEARN.METRICS F1:    ", f1_score_cv3_micro, "|", f1_score_cv3_macro, "|",
                                             f1_score_cv3_weighted)

  1%|█                                                                                 | 2/149 [00:04<05:24,  2.21s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  UserWarning,
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\i

 38%|██████████████████████████████▍                                                  | 56/149 [01:37<02:32,  1.64s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Us

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 48%|███████████████████████████████████████▏                                         | 72/149 [02:06<02:18,  1.80s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Us

 76%|████████████████████████████████████████████████████████████▋                   | 113/149 [03:16<00:47,  1.32s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Us

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 85%|████████████████████████████████████████████████████████████████████▏           | 127/149 [03:40<00:41,  1.87s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Us

 99%|███████████████████████████████████████████████████████████████████████████████▍| 148/149 [04:15<00:01,  1.62s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV SKLEARN.METRICS RMSE:     0.07629694974639094
CV SKLEARN.METRICS MAE :     0.008387458824004837
CV SKLEARN.METRICS MAP:     0.21839051259221648
CV SKLEARN.METRICS NDCG:     0.4806385904228656
CV SKLEARN.METRICS PRECISION:     0.9907644610669735 | 0.6462401262914017 | 0.9917727666182423
CV SKLEARN.METRICS RECALL:     0.9907644610669735 | 0.6638260471423343 | 0.9907644610669735
CV SKLEARN.METRICS F1:     0.9907644610669735 | 0.654915052750933 | 0.9912683574337822


In [14]:
print("TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-")

TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-


In [15]:
#UCENJE I PREDVIDANJE

from sklearn.tree import DecisionTreeClassifier
y_probabilities_dict={}

preds_test_series_dict={}
j=1
for playlist in tqdm(playlist_tracks_df_dict):
    clf = DecisionTreeClassifier(max_depth=30)
    clf.fit(X_train_dict[playlist], y_train_dict[playlist])
    y_probabilities = clf.predict_proba(X_test_dict[playlist])
    
    preds_test_series_dict[playlist] = pd.DataFrame(columns=['true', 'prediction'])
    preds_eval_list=[]
    for i in range(len(y_probabilities)):
        preds_eval_list.append(y_probabilities[i][1]) 
        
    preds_test_series_dict[playlist]['true'] = y_test_dict[playlist]
    preds_test_series_dict[playlist]['prediction'] = pd.Series(preds_eval_list).values


100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [00:35<00:00,  4.16it/s]


In [16]:
rmse_list_tt=[]
mae_list_tt=[]
ndcg_list_tt=[]
map_list_tt=[]

precision_list_tt_micro=[]
precision_list_tt_macro=[]
precision_list_tt_weighted=[]

recall_list_tt_micro=[]
recall_list_tt_macro=[]
recall_list_tt_weighted=[]

precision_list_tt_1=[]
recall_list_tt_1=[]
for playlist in tqdm(playlist_tracks_df_dict):
    if playlist!='playlist_tracks_df193':
        if playlist!='playlist_tracks_df193':
            rmse_list_tt.append(sqrt(mean_squared_error(preds_test_series_dict[playlist]['prediction'],
                                                        preds_test_series_dict[playlist]['true'])))
            mae_list_tt.append((mean_absolute_error(preds_test_series_dict[playlist]['prediction'],
                                                        preds_test_series_dict[playlist]['true'])))
            ndcg_list_tt.append(ndcg_score(preds_test_series_dict[playlist]['true'].values.reshape(1, -1),
                                           preds_test_series_dict[playlist]['prediction'].values.reshape(1, -1)))
            x=(average_precision_score(preds_test_series_dict[playlist]['true'].values,
                                       preds_test_series_dict[playlist]['prediction'].values))
            if not math.isnan(x):
                map_list_tt.append(x)

            df=preds_test_series_dict[playlist].copy()
            df.loc[df['prediction']>=average_100th, 'prediction']=1
            df.loc[df['prediction']<average_100th, 'prediction']=0

            precision_list_tt_micro.append(precision_score(df['true'], df['prediction'], average='micro'))
            precision_list_tt_macro.append(precision_score(df['true'], df['prediction'], average='macro'))
            precision_list_tt_weighted.append(precision_score(df['true'], df['prediction'], average='weighted'))

            recall_list_tt_micro.append(recall_score(df['true'], df['prediction'], average='micro'))
            recall_list_tt_macro.append(recall_score(df['true'], df['prediction'], average='macro'))
            recall_list_tt_weighted.append(recall_score(df['true'], df['prediction'], average='weighted'))

average_rmse = sum(rmse_list_tt) / len(rmse_list_tt)
average_mae = sum(mae_list_tt) / len(mae_list_tt)
average_ndcg = sum(ndcg_list_tt) / len(ndcg_list_tt)
average_map = sum(map_list_tt) / len(map_list_tt)

precision_tt_micro=np.mean(precision_list_tt_micro)
precision_tt_macro=np.mean(precision_list_tt_macro)
precision_tt_weighted=np.mean(precision_list_tt_weighted)

recall_tt_micro=np.mean(recall_list_tt_micro)
recall_tt_macro=np.mean(recall_list_tt_macro)
recall_tt_weighted=np.mean(recall_list_tt_weighted)

f1_tt_micro=(2*precision_tt_micro*recall_tt_micro)/(precision_tt_micro+recall_tt_micro)
f1_tt_macro=(2*precision_tt_macro*recall_tt_macro)/(precision_tt_macro+recall_tt_macro)
f1_tt_weighted=(2*precision_tt_weighted*recall_tt_weighted)/(precision_tt_weighted+recall_tt_weighted)

print('TRAIN-TEST SKLEARN.METRICS RMSE: ' + str(average_rmse))
print('TRAIN-TEST SKLEARN.METRICS MAE: ' + str(average_mae))
print('TRAIN-TEST SKLEARN.METRICS NDCG: ' + str(average_ndcg))
print('TRAIN-TEST SKLEARN.METRICS MAP: ' + str(average_map))
print('TRAIN-TEST SKLEARN.METRICS PRECISION: ',precision_tt_micro ,"|" ,precision_tt_macro ,"|" ,precision_tt_weighted )
print('TRAIN-TEST SKLEARN.METRICS RECALL: ' ,recall_tt_micro ,"|" ,recall_tt_macro ,"|" ,recall_tt_weighted )
print("TRAIN-TEST SKLEARN.METRICS F1:   ", f1_tt_micro ,"|" ,f1_tt_macro ,"|" ,f1_tt_weighted )

  0%|                                                                                          | 0/149 [00:00<?, ?it/s]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 14%|███████████▍                                                                     | 21/149 [00:00<00:03, 35.53it/s]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864

TRAIN-TEST SKLEARN.METRICS RMSE: 0.10011377586296405
TRAIN-TEST SKLEARN.METRICS MAE: 0.013639796485277344
TRAIN-TEST SKLEARN.METRICS NDCG: 0.27768075060036984
TRAIN-TEST SKLEARN.METRICS MAP: 0.022981862326205072
TRAIN-TEST SKLEARN.METRICS PRECISION:  0.984760409006665 | 0.5205088238886839 | 0.9877688206458936
TRAIN-TEST SKLEARN.METRICS RECALL:  0.984760409006665 | 0.525100991331426 | 0.984760409006665
TRAIN-TEST SKLEARN.METRICS F1:    0.984760409006665 | 0.522794823541641 | 0.9862623206801509
